Testet die Verlustfunktionen sowie ein Modell von variabler Länge

In [33]:
import tensorflow as tf

H = tf.constant([[1,2],
                 [2,3],
                 [3,4],
                 [4,5],
                 [5,6],
                ], dtype=float)

H = tf.reshape(H, (1,)+H.shape)

# dot product score
score = tf.matmul(H, H, transpose_b=True)

# scale score
dk = float(H.shape[-2])
score /= tf.sqrt(dk)

# softmax activation
A = tf.keras.activations.softmax(score, axis=-1)

# (deterministic) context vector
c_det = tf.matmul(A,H)

In [34]:
A

<tf.Tensor: shape=(1, 5, 5), dtype=float32, numpy=
array([[[3.4535166e-03, 1.3210785e-02, 5.0535418e-02, 1.9331384e-01,
         7.3948640e-01],
        [1.1653847e-04, 1.0903878e-03, 1.0202182e-02, 9.5456384e-02,
         8.9313453e-01],
        [3.4863342e-06, 7.9785830e-05, 1.8259232e-03, 4.1786812e-02,
         9.5630401e-01],
        [1.0003799e-07, 5.5997225e-06, 3.1344988e-04, 1.7545642e-02,
         9.8213518e-01],
        [2.8250795e-09, 3.8679164e-07, 5.2957017e-05, 7.2505325e-03,
         9.9269617e-01]]], dtype=float32)>

In [35]:
c_det

<tf.Tensor: shape=(1, 5, 2), dtype=float32, numpy=
array([[[4.6521688, 5.6521688],
        [4.880402 , 5.880402 ],
        [4.954308 , 5.954308 ],
        [4.98181  , 5.98181  ],
        [4.992643 , 5.9926424]]], dtype=float32)>

In [18]:
import numpy as np
import tensorflow as tf
x1 = np.array([[[1, 1], [2, 1], [3,1], [4,1]],
                 [[1.1, 1], [2.1, 1], [3.1,1], [4.1,1]],
                 [[1.2, 1], [2.2, 1], [3.2,1], [4.2,1]],
                ])
x2 = np.array([[[1, 1.1], [2, 1.1], [3,1.1], [5,1.1]],
                 [[1.1, 0], [2.1, 1], [3.1,1], [7.1,1]],
                 [[2.2, 1], [3.2, 1], [4.2,1], [5.2,1]],
                ])

mse = np.mean(((x1-x2)**2)**0.5, axis=1)
ansc = np.mean(mse, axis=-1)
loss = np.mean(ansc)

loss2 = tf.keras.losses.MeanAbsoluteError()(x1, x2)

loss3 = tf.keras.losses.MeanAbsoluteError(reduction=tf.keras.losses.Reduction.NONE)(x1, x2)

print(mse)
print(ansc)
print(loss)
print(loss2)
print(loss3)
print(x1.shape)

from tensorflow.keras import layers
class ReconstructionLossLayer(layers.Layer):

    def __init__(self, *args, **kwargs):
        super(ReconstructionLossLayer, self).__init__(*args, **kwargs)

    def call(self, inputs):
        x, x_hat =  inputs
        
        mse = tf.math.abs(tf.math.subtract(x, x_hat))
        mse = tf.math.reduce_mean(mse, axis=(1,2))
        
        return mse
    
from tensorflow import keras

inp1 = keras.Input(shape=x1.shape[1:])
inp2 = keras.Input(shape=x1.shape[1:])
out = ReconstructionLossLayer()([inp1, inp2])
mod = keras.Model(inputs =[inp1,inp2], outputs=out)
mod.compile()
result = mod.predict([x1,x2])

print(result)



[[0.25 0.1 ]
 [0.75 0.25]
 [1.   0.  ]]
[0.175 0.5   0.5  ]
0.39166666666666666
tf.Tensor(0.3916666507720947, shape=(), dtype=float64)
tf.Tensor(
[[0.05       0.05       0.05       0.55000001]
 [0.5        0.         0.         1.5       ]
 [0.5        0.5        0.5        0.5       ]], shape=(3, 4), dtype=float64)
(3, 4, 2)
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[0.17500001 0.5        0.49999997]


In [57]:
tf.keras.metrics.mean_squared_error(x1,x2)

TypeError: mean_squared_error() got an unexpected keyword argument 'axis'

In [121]:
from tensorflow import keras
from tensorflow.keras.layers import Layer

import numpy as np

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self):
        pass

    def __len__(self):
        'Denotes the number of batches per epoch'
        return 2

    def __getitem__(self, index):
        
        return np.ones((128,40,3)), np.ones((128,40,3))

    def on_epoch_end(self):
        pass

class MyLength(Layer):
    
    def __init__(self, *args, **kwargs):
        super(MyLength, self).__init__(*args, **kwargs)
    
    def call(self, inputs):
        
        s = tf.shape(inputs)[1]
        
        return inputs, s
    
class MyRepeat(Layer):
    
    def __init__(self, *args, **kwargs):
        super(MyRepeat, self).__init__(*args, **kwargs)
    
    def call(self, inputs):
        
        x, s = inputs
        
        x = tf.repeat(x, s, axis=1)
        
        return x

inp = keras.Input((None, 3))

x, s = MyLength()(inp)
x = keras.layers.LSTM(16)(x)
x = keras.layers.Dense(2)(x)

x = keras.layers.Dense(16)(x)
x = keras.layers.Reshape((1, 16))(x)
x = MyRepeat()([x,4])
#x = keras.layers.RepeatVector(40)(x)
out = keras.layers.LSTM(3, return_sequences=True)(x)

model = keras.Model(inputs=inp, outputs=out)
model.summary()

model.compile(optimizer="adam", loss="mse")

dg = DataGenerator()
                                           
                                           

model.fit(x=dg, epochs=10)


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "model_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_52 (InputLayer)        [(None, None, 3)]         0         
_________________________________________________________________
my_length_47 (MyLength)      ((None, None, 3), ())     0         
____________________________________________________